In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

In [3]:
x_data = np.array([[0,0],[0,1],[1,0],[1,1]], dtype = np.float32)
y_data = np.array([[0],[1],[1],[0]], dtype = np.float32) 

input_dim = len(x_data[0])
output_dim = len(y_data[0])

X = Variable(torch.tensor(x_data))
Y = Variable(torch.tensor(y_data))

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(input_dim, 2, bias = True)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(2, output_dim, bias = True)
        self.model = nn.Sequential(self.linear1, self.sigmoid, self.linear2, self.sigmoid)
        
    def forward(self, x):
        outputs = self.model(x)
        return outputs

    
    
    
model = Net()

In [8]:
lr = 0.1
optimiser = optim.SGD(model.parameters(), lr = lr)
criterion = nn.BCELoss(reduction = 'elementwise_mean')

for step in range(10001):
    optimiser.zero_grad()
    
    outputs = model.forward(X)
    cost = criterion(outputs, Y)
    cost.backward(retain_graph = True)
    optimiser.step()
    
    if step % 1000 == 0:
        print('step {}, loss {}'.format(step, cost.item()))

predicted = (model.forward(X) > 0.5).float()
accuracy = (predicted == Y).float().mean()

print('Prediction: ', predicted, '\n Accuracy: ', accuracy.data)
    

step 0, loss 0.6821415424346924
step 1000, loss 0.647502601146698
step 2000, loss 0.3645537197589874
step 3000, loss 0.11928139626979828
step 4000, loss 0.06166524067521095
step 5000, loss 0.04012984782457352
step 6000, loss 0.029344528913497925
step 7000, loss 0.02297881245613098
step 8000, loss 0.018814286217093468
step 9000, loss 0.015891965478658676
step 10000, loss 0.013735033571720123
Prediction:  tensor([[0.],
        [1.],
        [1.],
        [0.]]) 
 Accuracy:  tensor(1.)


## Wide NN for XOR

In [13]:
class WideNet(nn.Module):
    def __init__(self):
        super(WideNet, self).__init__()
        self.linear1 = nn.Linear(input_dim, 10, bias = True)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(10, output_dim, bias = True)
        self.model = nn.Sequential(self.linear1, self.sigmoid, self.linear2, self.sigmoid)
        
    def forward(self, x):
        outputs = self.model(x)
        return outputs

In [14]:
model2 = WideNet()

In [21]:
lr = 0.1
optimiser = optim.SGD(model2.parameters(), lr = lr)
criterion = nn.BCELoss(reduction = 'elementwise_mean')

for step in range(10001):
    optimiser.zero_grad()
    
    outputs = model2.forward(X)
    cost = criterion(outputs, Y)
    cost.backward(retain_graph = True)
    optimiser.step()
    
    if step % 1000 == 0:
        print('step {}, loss {}'.format(step, cost.item()))

predicted = model2.forward(X)
correct = (model2.forward(X) > 0.5).float()
accuracy = (correct == Y).float().mean()

print('Prediction: ', predicted, '\n Accuracy: ', accuracy.data)
    

step 0, loss 0.013081904500722885
step 1000, loss 0.011068901047110558
step 2000, loss 0.009569764137268066
step 3000, loss 0.008413081988692284
step 4000, loss 0.007495297119021416
step 5000, loss 0.006750427186489105
step 6000, loss 0.006134524941444397
step 7000, loss 0.005617176648229361
step 8000, loss 0.005176905542612076
step 9000, loss 0.00479782372713089
step 10000, loss 0.004468213766813278
Prediction:  tensor([[0.0042],
        [0.9946],
        [0.9964],
        [0.0047]], grad_fn=<SigmoidBackward>) 
 Accuracy:  tensor(1.)


## Deep NN for XOR

In [16]:
class DeepNet(nn.Module):
    def __init__(self):
        super(DeepNet, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.linear1 = nn.Linear(input_dim, 10, bias = True)
        self.linear2 = nn.Linear(10, 10, bias = True)
        self.linear3 =  nn.Linear(10, 10, bias = True)
        self.linear4 = nn.Linear(10,output_dim, bias = True)
        self.model = nn.Sequential(self.linear1, self.sigmoid, 
                                   self.linear2, self.sigmoid,
                                   self.linear3, self.sigmoid,
                                   self.linear4, self.sigmoid,)

    def forward(self, x):
        outputs = self.model(x)
        return outputs

In [17]:
model3 = DeepNet()

In [26]:
lr = 5
optimiser = optim.SGD(model3.parameters(), lr = lr)
criterion = nn.BCELoss(reduction = 'elementwise_mean')

for step in range(10001):
    optimiser.zero_grad()
    
    outputs = model3.forward(X)
    cost = criterion(outputs, Y)
    cost.backward(retain_graph = True)
    optimiser.step()
    
    if step % 1000 == 0:
        print('step {}, loss {}'.format(step, cost.item()))

predicted = model3.forward(X)
correct = (model3.forward(X) > 0.5).float()
accuracy = (correct == Y).float().mean()

print('Prediction: ', predicted, '\n Accuracy: ', accuracy.data)

step 0, loss 0.692841649055481
step 1000, loss 0.4790301024913788
step 2000, loss 0.0006271703750826418
step 3000, loss 0.00030078564304858446
step 4000, loss 0.00019702990539371967
step 5000, loss 0.00014620722504332662
step 6000, loss 0.00011613228707574308
step 7000, loss 9.625178063288331e-05
step 8000, loss 8.215387788368389e-05
step 9000, loss 7.163275586208329e-05
step 10000, loss 6.349610339384526e-05
Prediction:  tensor([[0.0000],
        [0.9999],
        [0.9999],
        [0.0000]], grad_fn=<SigmoidBackward>) 
 Accuracy:  tensor(1.)
